In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from plot_utils import plot_surface
import QAOA_paper_proxy

%matplotlib widget

def test_plot_number_with_cost(num_constraints: int, num_qubits: int) -> None:
    costs = np.linspace(0, num_constraints, num_constraints+1)
    num_with_costs = np.zeros(len(costs))
    for i in range (len(costs)):
        num_with_costs[i] = QAOA_paper_proxy.number_with_cost_paper_proxy(costs[i], num_constraints, num_qubits)
    
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.scatter(costs, num_with_costs)
    
def test_plot_by_cost_2(cost: int, num_constraints: int, num_qubits: int) -> None:
    cost_2s = np.linspace(0, num_constraints, num_constraints+1, dtype=int)
    distances = np.linspace(0, num_qubits, num_qubits+1, dtype=int)
    num_of_costs_at_distances = np.zeros([len(cost_2s), len(distances)])
    for i in range(len(cost_2s)):
        for j in range(len(distances)):
            num_of_costs_at_distances[i][j] = QAOA_paper_proxy.number_of_costs_at_distance_paper_proxy(cost, cost_2s[i], distances[j], num_constraints, num_qubits)
    
    plot_surface(cost_2s, distances, num_of_costs_at_distances, f"\"Approximate\" N({cost};d,c) from paper", "cost_2", "distance", "proxy")

test_plot_number_with_cost(20, 10)
test_plot_by_cost_2(4, 50, 12)
plt.show()

In [ ]:
from QAOA_paper_proxy import QAOA_paper_proxy
from plot_utils import plot_heat_map
import matplotlib.pyplot as plt
import numpy as np


def collect_parameter_data(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_paper_proxy(1, gamma, beta, num_constraints, num_qubits)

    return expectations


gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
overlaps = collect_parameter_data(10, 6, gammas, betas)
plot_heat_map(gammas, betas, overlaps, "Expectation Proxy from paper", "Gamma", "Beta")

plt.show()

In [ ]:
from QAOA_proxy import QAOA_proxy
from plot_utils import plot_heat_map
import matplotlib.pyplot as plt
import numpy as np


def collect_parameter_data(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray):
    expectations = np.zeros([len(gammas), len(betas)])
    amplitudes = np.zeros([num_constraints + 1, len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            amps, expectations[i][j] = QAOA_proxy(1, gamma, beta, num_constraints, num_qubits)
            for c in range(num_constraints+1):
                amplitudes[c][i][j] = abs(amps[1][c]) ** 2

    return amplitudes, expectations


gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
amplitudes, overlaps = collect_parameter_data(21, 10, gammas, betas)

for c in range(0, 21):
    plot_heat_map(gammas, betas, amplitudes[c], f"Amplitude for c={c}", "Gamma", "Beta")

plot_heat_map(gammas, betas, overlaps, f"Expectation Proxy", "Gamma", "Beta")

plt.show()

In [ ]:
from QAOA_paper_proxy import QAOA_paper_proxy
from plot_utils import plot_heat_map
import matplotlib.pyplot as plt
import numpy as np


def collect_parameter_data(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray):
    expectations = np.zeros([len(gammas), len(betas)])
    amplitudes = np.zeros([num_constraints + 1, len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            amps, expectations[i][j] = QAOA_paper_proxy(1, gamma, beta, num_constraints, num_qubits)
            for c in range(num_constraints+1):
                amplitudes[c][i][j] = abs(amps[1][c]) ** 2

    return amplitudes, expectations


gammas = np.linspace(0, np.pi/2, 25)
betas = np.linspace(0, np.pi, 25)
amplitudes, overlaps = collect_parameter_data(21, 10, gammas, betas)

for c in range(0, 21):
    plot_heat_map(gammas, betas, amplitudes[c], f"Amplitude for c={c}", "Gamma", "Beta")

plot_heat_map(gammas, betas, overlaps, f"Expectation Proxy", "Gamma", "Beta")

plt.show()

In [ ]:
from QAOA_simulator import get_simulator, get_overlap, get_result
from plot_utils import plot_heat_map
import networkx as nx
import qokit.maxcut as mc
import numpy as np

def collect_parameter_data(G: nx.Graph, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    overlaps = np.zeros([len(gammas), len(betas)])
    N = G.number_of_nodes()
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            result = get_result(N, ising_model, gamma, beta, sim)
            overlaps[i][j] = get_overlap(N, ising_model, gamma, beta, sim, result)

    return overlaps

N = 10
G = nx.erdos_renyi_graph(N, 0.5, seed=42)  # has 21 edges
gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
overlaps = collect_parameter_data(G, gammas, betas)
_ = plot_heat_map(gammas, betas, overlaps, "True QAOA Overlap", "Gamma", "Beta")

In [ ]:
from QAOA_simulator import get_simulator, get_expectation, get_result
from plot_utils import plot_heat_map
import networkx as nx
import qokit.maxcut as mc
import numpy as np

def collect_parameter_data(G: nx.Graph, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])
    N = G.number_of_nodes()
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            result = get_result(N, ising_model, gamma, beta, sim)
            expectations[i][j] = get_expectation(N, ising_model, gamma, beta, sim, result)

    return expectations

N = 10
G = nx.erdos_renyi_graph(N, 0.5, seed=42)  # has 21 edges
gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
overlaps = collect_parameter_data(G, gammas, betas)
_ = plot_heat_map(gammas, betas, overlaps, "True Expectation", "Gamma", "Beta")

In [ ]:
from QAOA_simulator import get_simulator, get_expectation, get_overlap, get_result
from plot_utils import plot_heat_map
import networkx as nx
import qokit.maxcut as mc
import numpy as np

def collect_parameter_data(G: nx.Graph, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])
    overlaps = np.zeros([len(gammas), len(betas)])
    N = G.number_of_nodes()
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            result = get_result(N, ising_model, gamma, beta, sim)
            expectations[i][j] = get_expectation(N, ising_model, gamma, beta, sim, result)
            overlaps[i][j] = get_overlap(N, ising_model, gamma, beta, sim, result)


    return expectations, overlaps

N = 15
G = nx.erdos_renyi_graph(N, 0.5, seed=42)  # has 21 edges
gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
expectations, overlaps = collect_parameter_data(G, gammas, betas)
shifted_expectations = expectations - np.amin(expectations)
_ = plot_heat_map(gammas, betas, expectations, "True Expectation", "Gamma", "Beta")
_ = plot_heat_map(gammas, betas, overlaps, "True Overlap", "Gamma", "Beta")
_ = plot_heat_map(gammas, betas, (shifted_expectations/np.amax(shifted_expectations)) - (overlaps/np.amax(overlaps)), "True Expectation Minus Overlap", "Gamma", "Beta")

In [ ]:
from QAOA_simulator import get_simulator, get_overlap, get_expectation, get_result
from plot_utils import plot_heat_map
import networkx as nx
import qokit.maxcut as mc
import numpy as np
from QAOA_proxy import QAOA_proxy
from QAOA_paper_proxy import QAOA_paper_proxy


def collect_parameter_data(G: nx.Graph, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])
    overlaps = np.zeros([len(gammas), len(betas)])
    N = G.number_of_nodes()
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            result = get_result(N, ising_model, gamma, beta, sim)
            overlaps[i][j] = get_overlap(N, ising_model, gamma, beta, sim, result)
            expectations[i][j] = get_expectation(N, ising_model, gamma, beta, sim, result)

    return expectations, overlaps


def collect_parameter_data_proxy_less_dumb(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray, h: float, hc: float, l: float, r: float) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])
    altered_proxy = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_proxy(1, gamma, beta, num_constraints, num_qubits, h, hc, l, r)
            _, altered_proxy[i][j] = QAOA_proxy(1, gamma, beta, num_constraints, num_qubits, h, hc, l, r, 2*(num_constraints+1)//3)

    return expectations, altered_proxy


def collect_parameter_data_proxy(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_paper_proxy(1, gamma, beta, num_constraints, num_qubits, 2*(num_constraints+1)//3)

    return expectations


N = 10
G = nx.erdos_renyi_graph(N, 0.5, seed=42)  # has 10 edges
gammas = np.linspace(0, np.pi, 50)
betas = np.linspace(0, 2*np.pi, 50)
expectations, overlaps = collect_parameter_data(G, gammas, betas)
_ = plot_heat_map(gammas, betas, expectations, "True QAOA Expectations", "Gamma", "Beta")
_ = plot_heat_map(gammas, betas, overlaps, "True QAOA Overlap", "Gamma", "Beta")
#expectation_proxies = collect_parameter_data_proxy(G.number_of_edges(), N, gammas, betas)
#_ = plot_heat_map(gammas, betas, expectation_proxies, "Expectation Proxies from Paper", "Gamma", "Beta")
expectation_proxies_less_dumb, altered_less_dumb = collect_parameter_data_proxy_less_dumb(G.number_of_edges(), N, gammas, betas, 0, 0, 1, 1)
_ = plot_heat_map(gammas, betas, expectation_proxies_less_dumb, "Expectation Proxies from us", "Gamma", "Beta")
_ = plot_heat_map(gammas, betas, altered_less_dumb, "Altered Proxies from us", "Gamma", "Beta")

In [ ]:
import QAOA_proxy as qpld
import numpy as np


def test_plot(num_constraints: int, num_qubits: int) -> None:
    costs = np.linspace(0, num_constraints, num_constraints + 1)
    num_with_costs = np.zeros(len(costs))
    for i in range(len(costs)):
        num_with_costs[i] = qpld.number_with_cost_proxy(costs[i], num_constraints, num_qubits)

    fig = plt.figure()
    ax = fig.add_subplot()
    ax.scatter(costs, num_with_costs)


test_plot(20, 10)

In [ ]:
import QAOA_proxy
import numpy as np
from plot_utils import plot_surface, animate_3d_rotation
import QAOA_paper_proxy
import matplotlib.animation as animation

%matplotlib widget

def test_plot_by_cost_paper(cost: int, num_constraints: int, num_qubits: int):
    cost_2s = np.linspace(0, num_constraints, num_constraints+1, dtype=int)
    distances = np.linspace(0, num_qubits, num_qubits+1, dtype=int)
    num_of_costs_at_distances = np.zeros([len(cost_2s), len(distances)])
    for i in range(len(cost_2s)):
        for j in range(len(distances)):
            num_of_costs_at_distances[i][j] = QAOA_paper_proxy.number_of_costs_at_distance_paper_proxy(cost, cost_2s[i], distances[j], num_constraints, num_qubits)
    
    return plot_surface(cost_2s, distances, num_of_costs_at_distances, f"\"Approximate\" N({cost};d,c) from paper", "cost_2", "distance", "proxy")

def test_plot_by_cost(cost: int, num_constraints: int, num_qubits: int) -> None:
    cost_2s = np.linspace(0, num_constraints, num_constraints+1, dtype=int)
    distances = np.linspace(0, num_qubits, num_qubits+1, dtype=int)
    num_of_costs_at_distances = np.zeros([len(cost_2s), len(distances)])
    for i in range(len(cost_2s)):
        for j in range(len(distances)):
            num_of_costs_at_distances[i][j] = QAOA_proxy.number_of_costs_at_distance_proxy(cost, cost_2s[i], distances[j], num_constraints, num_qubits)
    
    return plot_surface(cost_2s, distances, num_of_costs_at_distances, f"\"Approximate\" N({cost};d,c) from us", "cost_2", "distance", "proxy")

fig, ax = test_plot_by_cost_paper(8, 20, 10)
fig2, ax2 = test_plot_by_cost(8, 20, 10)

animate_3d_rotation(fig, ax, 'rotation1.gif')
animate_3d_rotation(fig2, ax2, 'rotation2.gif')

In [ ]:
from QAOA_paper_proxy import number_of_costs_at_distance_paper_proxy
import numpy as np
from plot_utils import plot_3d_scatter

%matplotlib widget

def test_plot(num_constraints_max: int, num_qubits_max: int) -> None:
    num_constraints = np.linspace(0, num_constraints_max, num_constraints_max + 1, dtype=int)
    num_qubits = np.linspace(0, num_qubits_max, num_qubits_max + 1, dtype=int)
    peak_hs = np.zeros([len(num_constraints), len(num_qubits)])
    for c in range(len(num_constraints)):
        for q in range(len(num_qubits)):
            try:
                peak_hs[c][q] = number_of_costs_at_distance_paper_proxy(c//2, c//2, q//2, c, q)
            except:
                ()
    
    plot_3d_scatter(num_constraints[7:], num_qubits, peak_hs[7:], f"Peak values from Paper", "num_constraints", "num_qubits", "peak value")

test_plot(15, 12)

In [ ]:
import numpy as np
from plot_utils import plot_3d_scatter

%matplotlib widget

def test_plot(num_constraints_max: int, num_qubits_max: int) -> None:
    num_constraints = np.linspace(0, num_constraints_max, num_constraints_max + 1, dtype=int)
    num_qubits = np.linspace(0, num_qubits_max, num_qubits_max + 1, dtype=int)
    peak_hs = np.zeros([len(num_constraints), len(num_qubits)])
    for c in range(len(num_constraints)):
        for q in range(len(num_qubits)):
            try:
                peak_hs[c][q] = 2**(q-4)
            except:
                ()
    plot_3d_scatter(num_constraints, num_qubits, peak_hs, f"Peak values from us", "num_constraints", "num_qubits", "peak value_proxy")

test_plot(15, 12)

In [ ]:
import networkx as nx

N = 10
nx.erdos_renyi_graph(N, 0.5, seed=42).number_of_edges()

In [1]:
from QAOA_simulator import get_expectation, get_simulator, get_overlap, QAOA_run
import networkx as nx
import qokit.maxcut as mc
import numpy as np
import QAOA_proxy as qp
import QAOA_proxy_optimizer as qpo
#import QAOA_paper_proxy as qpp
from itertools import product


N = 10
p = 2
G = nx.erdos_renyi_graph(N, 0.5, seed = 42)
num_edges = G.number_of_edges()
ising_model = mc.get_maxcut_terms(G)
sim = get_simulator(N, ising_model)
print(f"Max Cut: {max(sim.get_cost_diagonal())}")
init_gamma, init_beta = np.full((2, p), 0.1)

real_result = QAOA_run(ising_model, N, p, init_gamma, init_beta)
real_gamma = real_result["gamma"]
real_beta = real_result["beta"]
real_expectation = real_result["expectation"]
real_overlap = real_result["overlap"]
print(f"g: {real_gamma}, b: {real_beta}, e: {real_expectation}, o: {real_overlap}")

pre_result = qp.QAOA_proxy_run(num_edges, N, p, init_gamma, init_beta)
pre_gamma = pre_result["gamma"]
pre_beta = pre_result["beta"]
pre_expectation = get_expectation(N, ising_model, pre_gamma, pre_beta)
pre_overlap = get_overlap(N, ising_model, pre_gamma, pre_beta)
print(f"g: {pre_gamma}, b: {pre_beta}, e: {pre_expectation}, o: {pre_overlap}")

init_h_tweaks = [1]
init_hc_tweaks = [0, 2]
init_l_tweaks = [0.1, 1, 2]
init_r_tweaks = [0.1, 1, 2]
init_tweaks = product(init_h_tweaks, init_hc_tweaks, init_l_tweaks, init_r_tweaks)

h, hc, l, r = qpo.QAOA_proxy_optimize(num_edges, N, p, init_gamma, init_beta, init_tweaks=init_tweaks)
print(f"h: {h}\nhc: {hc}\nl: {l}\nr: {r}")
result = qp.QAOA_proxy_run(num_edges, N, p, init_gamma, init_beta, h_tweak_sub=h, hc_tweak_add=hc, l_tweak_mul=l, r_tweak_mul=r)
gamma = result["gamma"]
beta = result["beta"]
expectation = get_expectation(N, ising_model, gamma, beta)
approx_ratio = expectation/max(sim.get_cost_diagonal())
overlap = get_overlap(N, ising_model, gamma, beta)
print(f"g: {gamma}, b: {beta}, e: {expectation}, o: {overlap}")

print("Start")
for _ in range(10):
    G = nx.erdos_renyi_graph(N, 0.5)
    while (G.number_of_edges() != num_edges):
        G = nx.erdos_renyi_graph(N, 0.5)
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)
    print(f"MaxCut: {max(sim.get_cost_diagonal())}")
    print(f"e: {get_expectation(N, ising_model, gamma, beta, sim)}, o: {get_overlap(N, ising_model, gamma, beta, sim)}")


#result2 = qpp.QAOA_paper_proxy_run(G.number_of_edges(), N, p, init_gamma, init_beta)
#gamma2 = result2["gamma"]
#beta2 = result2["beta"]
#expectation2 = get_expectation(N, ising_model, gamma2, beta2)
#print(f"g: {gamma2}, b: {beta2}, e: {expectation2}")

/home/nkohen/dev/QOKit/grips/QAOA_proxy.py:26: JuliaWarning: USE_JULIA=False, so calling python version of QAOA_proxy. Did you mean to call QAOA_proxy_python?
  warnings.warn("USE_JULIA=False, so calling python version of QAOA_proxy. Did you mean to call QAOA_proxy_python?", JuliaWarning)


Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython
Max Cut: 15.0
g: [0.60729518 1.23338023 1.51375319], b: [0.4580484  0.3055645  0.16623899], e: 14.230756860040874, o: 0.5042338171708334
g: [ 0.73097805 -1.23440413 -0.16696553], b: [1.94927137 1.19545912 1.1864342 ], e: 8.160270598645209, o: 0.009908793618320278
h: 1.9682688294810495
hc: 2.749016430460764
l: 1.829672431219189
r: 1.2590326485080765
g: [-0.53786195 -0.07879607  0.87280154], b: [1.2226664  1.22464704 1.92107174], e: 12.764466884853158, o: 0.23203200693946163
Start
MaxCut: 15.0
e: 12.653307664275037, o: 0.1531939914474503
MaxCut: 16.0
e: 13.096239534551717, o: 0.15835161216874102
MaxCut: 16.0
e: 12.640448563126089, o: 0.08046690432958996
MaxCut: 17.0
e: 12.908129585173207, o: 0.07149563920101255
MaxCut: 17.0
e: 12.963392677437799, o: 0.07876652679623507
MaxCut: 16.0
e: 12.86229569696382, o: 0.05167833392757955
MaxCut: 16.0
e: 12.356335673639405, o: 0.049483823

In [9]:
p = 2
init_gamma, init_beta = np.full((2, p), 0.1)

max_N = 8
min_N = 5

init_tweaks = []
results = dict()

for N in range(min_N, max_N + 1):
    max_edges = N*(N-1)//2
    if (N == min_N):
        init_tweaks = [(0, 0, 1, 1)]
    elif (N-1, max_edges // 3) in results:
        init_tweaks = [results[N-1, max_edges // 3]]
    elif (N-1, (max_edges // 3) - 1) in results:
        init_tweaks = [results[N-1, (max_edges // 3) - 1]]
    elif (N-1, (max_edges // 3) - 2) in results:
        init_tweaks = [results[N-1, (max_edges // 3) - 2]]
    for num_edges in range(max_edges // 3, max_edges - 3, 3):
        h, hc, l, r = qpo.QAOA_proxy_optimize(num_edges, N, p, init_gamma, init_beta, init_tweaks=init_tweaks)
        results[N, num_edges] = (h, hc, l, r)
        init_tweaks = [(h, hc, l, r)]

print(results)

{(5, 3): (-0.8695889123346713, -0.7508806016227224, -5.455705331947058e-20, -5.4210108624275204e-20), (5, 6): (-0.12015673236921685, -0.7549867697049777, 0.0008611360944436513, 0.2067134780801421), (5, 9): (-0.1684519235890042, -0.7568195043747276, 0.01005555884740752, 1.2254383864242362), (6, 5): (0.03755465570957784, -0.6406813431589818, 5.612432962440333e-05, 0.10115882164397375), (6, 8): (0.9515623920973388, -0.9154778296781629, 0.0006652872944667141, 0.2992432017434516), (6, 11): (1.7525735385514107, -1.0017084070214899, 0.012752210986596615, 0.41165166679690895), (6, 14): (2.0738151571268095, -0.7808966996062686, 6.569957000242946e-05, 0.8832524368486849), (7, 7): (2.506042726958915, -0.5966382499499451, 6.64618136330888e-06, 1.7833387852856833e-05), (7, 10): (3.4364922142881564, -0.5930029329286116, 0.051570215461367704, 0.2889187994695418), (7, 13): (4.415746690331215, -0.5326948582371996, 0.005329199836494475, 0.306636162722376), (7, 16): (4.509473812434786, -0.476097992257600

In [31]:
from sklearn import linear_model
import numpy as np
x = np.array(list(results.keys()))
y = np.array(list(results.values()))
print(x)
print()
print(y)
regr = linear_model.LinearRegression().fit(x, y)
print()
print(regr.coef_)
print(regr.score(x, y))
regr.predict(np.array([[8, 21]]))

[[ 5  3]
 [ 5  6]
 [ 5  9]
 [ 6  5]
 [ 6  8]
 [ 6 11]
 [ 6 14]
 [ 7  7]
 [ 7 10]
 [ 7 13]
 [ 7 16]
 [ 7 19]
 [ 8  9]
 [ 8 12]
 [ 8 15]
 [ 8 18]
 [ 8 21]
 [ 8 24]
 [ 8 27]]

[[-8.69588912e-01 -7.50880602e-01 -5.45570533e-20 -5.42101086e-20]
 [-1.20156732e-01 -7.54986770e-01  8.61136094e-04  2.06713478e-01]
 [-1.68451924e-01 -7.56819504e-01  1.00555588e-02  1.22543839e+00]
 [ 3.75546557e-02 -6.40681343e-01  5.61243296e-05  1.01158822e-01]
 [ 9.51562392e-01 -9.15477830e-01  6.65287294e-04  2.99243202e-01]
 [ 1.75257354e+00 -1.00170841e+00  1.27522110e-02  4.11651667e-01]
 [ 2.07381516e+00 -7.80896700e-01  6.56995700e-05  8.83252437e-01]
 [ 2.50604273e+00 -5.96638250e-01  6.64618136e-06  1.78333879e-05]
 [ 3.43649221e+00 -5.93002933e-01  5.15702155e-02  2.88918799e-01]
 [ 4.41574669e+00 -5.32694858e-01  5.32919984e-03  3.06636163e-01]
 [ 4.50947381e+00 -4.76097992e-01  1.12113127e-02  1.29729265e+00]
 [ 5.78733964e+00 -1.16019172e+00  4.43028859e-04  6.34161685e-01]
 [ 4.33683481e+00  1.17

array([[6.47178606, 2.43121684, 0.44170764, 0.23009686]])

In [34]:
from QAOA_simulator import get_expectation, get_simulator, get_overlap
import QAOA_proxy as qp
import qokit.maxcut as mc
import Graph_util

p = 2
init_gamma, init_beta = np.full((2, p), 0.1)

max_N = 8
min_N = 5

for N in range(min_N, max_N + 1):
    max_edges = N*(N-1)//2
    for num_edges in range(max_edges // 3, max_edges - 3, 3):
        h, hc, l, r = results[N, num_edges]
        res = qp.QAOA_proxy_run(num_edges, N, p, init_gamma, init_beta, h_tweak_sub=h, hc_tweak_add=hc, l_tweak_mul=l, r_tweak_mul=r)
        gamma = res["gamma"]
        beta = res["beta"]
        guess = regr.predict(np.array([[N, num_edges]]))
        h_guess, hc_guess, l_guess, r_guess = guess[0][0], guess[0][1], guess[0][2], guess[0][3]
        res_guess = qp.QAOA_proxy_run(num_edges, N, p, init_gamma, init_beta, h_tweak_sub=h_guess, hc_tweak_add=hc_guess, l_tweak_mul=l_guess, r_tweak_mul=r_guess)
        gamma_guess = res_guess["gamma"]
        beta_guess = res_guess["beta"]
        for _ in range(10):
            G = Graph_util.erdos_renyi(N, num_edges)
            ising_model = mc.get_maxcut_terms(G)
            sim = get_simulator(N, ising_model)
            print(f"MaxCut ({N}, {num_edges}): {max(sim.get_cost_diagonal())}")
            print(f"Optimal: e: {get_expectation(N, ising_model, gamma, beta, sim)}, o: {get_overlap(N, ising_model, gamma, beta, sim)}")
            print(f"Guess: e: {get_expectation(N, ising_model, gamma_guess, beta_guess, sim)}, o: {get_overlap(N, ising_model, gamma_guess, beta_guess, sim)}")

MaxCut (5, 3): 2.0
Optimal: e: 1.6734543906794488, o: 0.8367271953397244
Guess: e: 1.5913960715466484, o: 0.7956980357733243
MaxCut (5, 3): 3.0
Optimal: e: 2.3181217556864233, o: 0.4739409537097374
Guess: e: 2.206696992515667, o: 0.4571425195520661
MaxCut (5, 3): 2.0
Optimal: e: 1.673454390679448, o: 0.836727195339724
Guess: e: 1.5913960715466482, o: 0.7956980357733241
MaxCut (5, 3): 3.0
Optimal: e: 2.381115612674786, o: 0.4674380114096777
Guess: e: 2.256202067750527, o: 0.42665929174219724
MaxCut (5, 3): 3.0
Optimal: e: 2.3181217556864233, o: 0.47394095370973727
Guess: e: 2.206696992515666, o: 0.4571425195520659
MaxCut (5, 3): 3.0
Optimal: e: 2.381115612674786, o: 0.4674380114096777
Guess: e: 2.256202067750529, o: 0.4266592917421976
MaxCut (5, 3): 3.0
Optimal: e: 2.3181217556864224, o: 0.4739409537097371
Guess: e: 2.206696992515667, o: 0.45714251955206625
MaxCut (5, 3): 3.0
Optimal: e: 2.3811156126747868, o: 0.4674380114096779
Guess: e: 2.256202067750528, o: 0.4266592917421976
MaxCut 